In [152]:
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option("display.max_columns", None)

In [48]:
df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)
df.head()

C:\Users\vojte\AppData\Local\Temp\ipykernel_30048\3437320206.py:1: DtypeWarning: Columns (65,76,91,154,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,GBH,GBD,GBA,IWH,IWD,IWA,SBH,SBD,SBA,WHH,WHD,WHA,country,league,season,SYH,SYD,SYA,B365H,B365D,B365A,GB>2.5,GB<2.5,GBAHH,GBAHA,GBAH,BWH,BWD,BWA,LBH,LBD,LBA,B365AHH,B365AHA,B365AH,SJH,SJD,SJA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,BSH,BSD,BSA,PSH,PSD,PSA,PSCH,PSCD,PSCA,HS,AS,HST,AST,HFKC,AFKC,HC,AC,HY,AY,HR,AR,Time,HF,AF,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Attendance,Referee,HHW,AHW,HO,AO,HBP,ABP,SOH,SOD,SOA,LBAHH,LBAHA,LBAH,LB,LB.1,LB.2,HT,AT
0,B1,20/05/01,Mechelen,Lierse,1.0,1.0,D,1.0,1.0,D,2.52,3.10,2.47,NaN,NaN,NaN,NaN,NaN,NaN,2.40,3.40,2.35,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B1,17/09/00,Germinal,Club Brugge,2.0,3.0,A,0.0,1.0,A,3.60,3.50,1.80,3.20,3.1,1.85,3.50,3.3,NaN,3.10,3.30,1.95,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B1,14/10/00,Beveren,Standard,0.0,3.0,A,0.0,2.0,A,3.30,3.40,1.85,3.40,3.2,1.75,3.20,3.4,NaN,3.25,3.45,1.85,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B1,06/09/00,Lokeren,Mechelen,3.0,2.0,H,0.0,2.0,A,1.66,3.50,3.95,1.55,3.3,4.50,1.67,3.5,NaN,1.70,3.50,3.80,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B1,27/08/00,Charleroi,Antwerp,1.0,0.0,H,1.0,0.0,H,1.60,3.35,4.80,1.60,3.3,4.00,1.62,3.5,NaN,1.60,3.40,4.50,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [232]:
df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)

bet_columns = ["B365", "BS", "BW", "GB", "IW", "LB", "PS", "SO", "SB", "SJ", "SY", "VC", "WH"]
home_bet = [col+"H" for col in bet_columns]
draw_bet = [col+"D" for col in bet_columns]
away_bet = [col+"A" for col in bet_columns]

def check_bet_columns(df):
    diff = set(home_bet) - set(df.columns)
    if len(diff)>0:
        print(f"There are {len(diff)} missing betting columns in the data.")
    else:
        print("All betting columns are present!")

df["AvgH"] = df[home_bet].mean(axis=1)
df["AvgD"] = df[draw_bet].mean(axis=1)
df["AvgA"] = df[away_bet].mean(axis=1)

check_bet_columns(df)

def dropnas(df):
    df = df.loc[~df["FTHG"].isna() & ~df["FTAG"].isna()].copy()
    df = df.loc[~df["HTHG"].isna() & ~df["HTAG"].isna()].copy()
    df = df.loc[~df["HomeTeam"].isna() & ~df["AwayTeam"].isna()].copy()
    df = df.loc[~df["AvgH"].isna() & ~df["AvgD"].isna() & ~df["AvgA"].isna()].copy()
    df = df.loc[~df["Div"].isna() & ~df["Date"].isna()].copy()
    df = df.loc[~df["FTR"].isna() & ~df["HTR"].isna()].copy()
    return df

def dropoutliers(df):
    df = df.loc[
        (df["FTHG"] >= 0) & (df["FTHG"] < 50) & (df["FTAG"] >= 0) & (df["FTAG"] < 50)
        ].copy()
    return df

def limit(val, lim):
    return val if val <= lim else lim

def capoutliers(df):
    df["ExtremH"] = df["AvgH"].apply(lambda x: x>10)
    df["AvgH"] = df["AvgH"].apply(lambda x: limit(x, 10))
    df["ExtremD"] = df["AvgD"].apply(lambda x: x>10)
    df["AvgD"] = df["AvgD"].apply(lambda x: limit(x, 10))
    df["ExtremA"] = df["AvgA"].apply(lambda x: x>10)
    df["AvgA"] = df["AvgA"].apply(lambda x: limit(x, 10))
    return df

def fill_goals(df):
    df["HTR"] = calculate_win(df["HTHG"], df["HTAG"])
    df["FTR_test"] = calculate_win(df["FTHG"], df["FTAG"])
    return df

def calculate_win(home_goals, away_goals):
    results = []
    for home, away in zip(home_goals, away_goals):
        if home < away:
            results.append("A")
        elif home > away:
            results.append("H")
        else:
            results.append("D")
    return results

def process_dates(df):
    df["Date"] = df["Date"].apply(
        lambda date: datetime.strptime(date, "%d/%m/%y") if len(date)==8 else datetime.strptime(date, "%d/%m/%Y")
    )
    return df

df = dropnas(df)
df = dropoutliers(df)
df = capoutliers(df)
df = fill_goals(df)
df = process_dates(df)

C:\Users\vojte\AppData\Local\Temp\ipykernel_30048\3291044820.py:1: DtypeWarning: Columns (65,76,91,154,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)


All betting columns are present!


In [236]:
df["DivAndYear"] = df["Date"].apply(lambda x: str(x.year)) + df["Date"].apply(lambda x: " ") + df["Div"]

In [237]:
from sklearn.model_selection import train_test_split

train, temp = train_test_split(df, test_size=0.3, stratify=df["DivAndYear"])
val, test = train_test_split(temp, test_size=0.5, stratify=temp["DivAndYear"])

In [238]:
teams_df = pd.concat([
    train.rename(
        {"HomeTeam": "Team", "FTHG": "FTG", "HTHG": "HTG", "HS": "Shots", "HST": "ShotsOnTarget", "HHW": "HitWoodwork", "HC": "Corners", "HF": "Fouls", "HFKC": "KicksConceded", "HO": "Offsides", "HY": "YellowCards", "HR": "RedCards"}
        ,axis=1)[["Team", "FTG", "HTG", "Shots", "ShotsOnTarget", "HitWoodwork", "Corners", "Fouls", "KicksConceded", "Offsides", "YellowCards", "RedCards"]],
    train.rename(
        {"AwayTeam": "Team", "FTAG": "FTG", "HTAG": "HTG", "AS": "Shots", "AST": "ShotsOnTarget", "AHW": "HitWoodwork", "AC": "Corners", "AF": "Fouls", "AFKC": "KicksConceded", "AO": "Offsides", "AY": "YellowCards", "AR": "RedCards"}
        ,axis=1)[["Team", "FTG", "HTG", "Shots", "ShotsOnTarget", "HitWoodwork", "Corners", "Fouls", "KicksConceded", "Offsides", "YellowCards", "RedCards"]]
])

In [239]:
avg_shots_by_team = teams_df.groupby("Team")["Shots"].mean().to_dict()
avg_shots = teams_df["Shots"].mean()
for key, value in avg_shots_by_team.items():
    if pd.isna(value):
        avg_shots_by_team[key] = avg_shots

# encode teams into average shot count
for df_choose in (train, test, val):
    df_choose["HomeTeamEnc"] = df_choose["HomeTeam"].apply(lambda x: avg_shots_by_team.get(x, avg_shots))
    df_choose["AwayTeamEnc"] = df_choose["AwayTeam"].apply(lambda x: avg_shots_by_team.get(x, avg_shots))


In [240]:
cards_df = pd.concat([
    train.rename(
        {"HY": "YellowCards", "HR": "RedCards"}
        ,axis=1)[["Div", "YellowCards", "RedCards"]],
    train.rename(
        {"AY": "YellowCards", "AR": "RedCards"}
        ,axis=1)[["Div", "YellowCards", "RedCards"]]
])

In [241]:
yellow_cards_by_div = cards_df.groupby("Div")["YellowCards"].mean().to_dict()
red_cards_by_div = cards_df.groupby("Div")["RedCards"].mean().to_dict()

# encode divisions into average card given count
for df_choose in (train, test, val):
    df_choose["DivisionEncYellow"] = df_choose["Div"].map(yellow_cards_by_div)
    df_choose["DivisionEncRed"] = df_choose["Div"].map(red_cards_by_div)

In [242]:
chosen_columns = [
    "FTHG", "FTAG", "FTR",
    "HTHG", "HTAG", "HTR", 
    "HS", "AS", "HST", "AST",
    "HF", "AF",
    "HY", "AY", "HR", "AR", 
    "AvgH", "AvgA", "AvgD", 
    "HomeTeamEnc", "AwayTeamEnc"
    ]

In [243]:
train[chosen_columns].to_csv("train_processed.csv")
test[chosen_columns].to_csv("test_processed.csv")
val[chosen_columns].to_csv("val_processed.csv")